In [82]:
import json
import pandas as pd
from collections import Counter
import seaborn as sns
from matplotlib.pyplot import plot, figure
#from bokeh.plotting import figure, output_file, show

In [25]:
with open("data/archive_programming.json", "r") as fp:
    final_data = json.load(fp)

In [28]:
keys_parent = [
    "creatorId", "homeCollectionId", "title", "detectedLanguage", "latestVersion", "latestPublishedVersion",
    "hasUnpublishedEdits", "latestRev", "createdAt", "updatedAt", "acceptedAt", "firstPublishedAt",
    "latestPublishedAt", "uniqueSlug", "isEligibleForRevenue"
]

keys_virtuals = [
    "imageCount", "readingTime", "subtitle", "recommends", "isBookmarked",
    "socialRecommendsCount", "responsesCreatedCount", "totalClapCount", "sectionCount"
]

keys_others = [
    "linkCount", "userId", "name", "username", "collectionName", "followerCount"
]

def is_keys_unique(*key_lists):
    all_keys = []
    for key_list in key_lists:
        if not isinstance(key_list, list):
            return (False, [])
        all_keys += key_list
        
    freq = Counter(all_keys)
    common_keys = [k for k,v in dict(freq).items() if v > 1]
        
    return (len(common_keys) == 0, common_keys)

def get_required_fields(artical, User, Collection):
    fields = {}
    for key in keys_parent:
        fields[key] = artical[key]
        
    virtuals = artical["virtuals"]
    for key in keys_virtuals:
        fields[key] = virtuals[key]
        
    links = virtuals.get("links", None)
    if links:
        fields["linkCount"] = len(links["entries"])
    else:
        fields["linkCount"] = 0
    #fields["userId"] = virtuals["userPostRelation"]["userId"]
    tags = virtuals.get("tags", None)
    if tags:
        tagList = [x["name"] for x in tags]
    else:
        tagList = []
        
    fields["tags"] = tagList
    
    fields["name"] = User[fields["creatorId"]]["name"]
    fields["username"] = User[fields["creatorId"]]["username"]
    
    if fields["homeCollectionId"] and Collection.get(fields["homeCollectionId"], None):
        fields["collectionName"] = Collection[fields["homeCollectionId"]]["name"]
        fields["followerCount"] = Collection[fields["homeCollectionId"]]["metadata"]["followerCount"]
    else:
        fields["collectionName"] = None
        fields["followerCount"] = None
        
    return fields

In [29]:
res, li = is_keys_unique(keys_parent, keys_virtuals, keys_others)
index = 0
if res:
    filtered_data = []
    for k, v in final_data["Post"].items():
        filtered = get_required_fields(v, final_data["User"], final_data["Collection"])
        filtered_data.append(filtered)
        index+=1
        #if index == 100:
        #    break
else:
    print("keys are not unique: ", li)

In [30]:
len(filtered_data)

136296

In [31]:
df = pd.DataFrame(filtered_data)

In [32]:
del filtered_data
del final_data

In [33]:
df.columns

Index(['creatorId', 'homeCollectionId', 'title', 'detectedLanguage',
       'latestVersion', 'latestPublishedVersion', 'hasUnpublishedEdits',
       'latestRev', 'createdAt', 'updatedAt', 'acceptedAt', 'firstPublishedAt',
       'latestPublishedAt', 'uniqueSlug', 'isEligibleForRevenue', 'imageCount',
       'readingTime', 'subtitle', 'recommends', 'isBookmarked',
       'socialRecommendsCount', 'responsesCreatedCount', 'totalClapCount',
       'sectionCount', 'linkCount', 'tags', 'name', 'username',
       'collectionName', 'followerCount'],
      dtype='object')

In [34]:
drops = ['latestVersion', 'latestPublishedVersion', 'hasUnpublishedEdits', 'latestRev', 'updatedAt',
        'acceptedAt', 'firstPublishedAt', 'latestPublishedAt', 'uniqueSlug', 'isEligibleForRevenue']

df.drop(drops, axis=1, inplace=True)

In [35]:
df[df["detectedLanguage"] == 'en'].shape

(125816, 20)

In [36]:
df["totalArticles"] = 1

In [37]:
df.loc[df["homeCollectionId"] == "", "collectionName"] = "individual"

In [38]:
#df_pie = df.groupby("collectionName").sum().sort_values(by="totalArticles", ascending=False)
#df_pie["other"] = df_pie.index
#df_pie.loc[df_pie["totalArticles"] < 100, "other"] = "Other"
#df_pie.groupby("other").sum().plot.pie(y="totalArticles", figsize=(15,15))

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136296 entries, 0 to 136295
Data columns (total 21 columns):
creatorId                136296 non-null object
homeCollectionId         136296 non-null object
title                    136296 non-null object
detectedLanguage         136296 non-null object
createdAt                136296 non-null int64
imageCount               136296 non-null int64
readingTime              136296 non-null float64
subtitle                 136296 non-null object
recommends               136296 non-null int64
isBookmarked             136296 non-null bool
socialRecommendsCount    136296 non-null int64
responsesCreatedCount    136296 non-null int64
totalClapCount           136296 non-null int64
sectionCount             136296 non-null int64
linkCount                136296 non-null int64
tags                     136296 non-null object
name                     136296 non-null object
username                 136296 non-null object
collectionName           136178 no

In [40]:
df.tail()

creatorId homeCollectionId  \
136291  89c0cec8b946                    
136292  fe0012478878                    
136293  466ea72e9fb2                    
136294  f7ec597ebb71                    
136295  3179e6d5894b                    

                                                    title detectedLanguage  \
136291  1 big secret about data storage which will hel...               en   
136292  So many times! Even just going to the toilet c...               en   
136293                                          程式語言-核心概念          zh-Hant   
136294                             Stack — Data Structure               en   
136295  Bumili ng mga Counterfeit US Dollars sa online...               tl   

            createdAt  imageCount  readingTime  \
136291  1579858961856           2     2.368239   
136292  1579858734501           0     0.056604   
136293  1579851318518           1     2.550000   
136294  1579810984272           1     1.211321   
136295  1579838585740           3     2.927358   

                                                 subtitle  recommends  \
136291  We define unstructured data as data with littl...           0   
136292                                                              1   
136293                        程式語言雖然分為很多種，但概念不外乎涵蓋以下六項元素：           0   
136294                                     What is Stack?           0   
136295  Bumili ng mga Counterfeit US Dollars sa online...           0   

        isBookmarked  ...  responsesCreatedCount  totalClapCount  \
136291         False  ...                      0               0   
136292         False  ...                      0              50   
136293         False  ...                      0               0   
136294         False  ...                      0               0   
136295         False  ...                      0               0   

        sectionCount  linkCount  \
136291             1          2   
136292             1          0   
136293             2          2   
136294             1          0   
136295             1          0   

                                                     tags  \
136291  [AI, Machine Learning, Programming, Data Scien...   
136292  [Programming, Learning, Productivity, Inspirat...   
136293                             [Programming, 程式語言, C]   
136294  [Stack, Data Structures, Programming, Java, So...   
136295  [Financial Planning, Design, Programming, Oppo...   

                       name           username collectionName followerCount  \
136291   El Akioui Zouhaire  zouhaire.elakioui     individual           NaN   
136292       Reece Metcalfe      reecemetcalfe     individual           NaN   
136293        Yu Zhuang Lin        yuzhuanglin     individual           NaN   
136294  Raja Aakash Rathore    raja.aakash2001     individual           NaN   
136295          Frankycanna       frankycanna1     individual           NaN   

        totalArticles  
136291              1  
136292              1  
136293              1  
136294              1  
136295              1  

[5 rows x 21 columns]

In [41]:
df['date'] = pd.to_datetime(df['createdAt'],unit='ms')

In [85]:
langs = ["Python", "JavaScript", "Cpp", "C", "Swift"]

In [68]:
def addLanguages(li):
    for el in li:
        df[el] = df["tags"].apply(lambda x: 1 if el in x else 0)
        print(el, ': ', (df[el] == 1).sum())

In [69]:
addLanguages(langs)

Programming :  134124
Python :  9190
JavaScript :  20072
Cpp :  810
C :  941
Swift :  3064


In [107]:
aggColumns = {'totalArticles':'mean'}#, 'totalClapCount':'sum'}
for lang in langs:
    aggColumns.update({lang: 'mean'})

In [108]:
new_df = (df.groupby([pd.Grouper(key='date', freq='1W')])
          .agg(aggColumns))

In [109]:
new_df.tail(10)

totalArticles    Python  JavaScript       Cpp         C     Swift
date                                                                         
2019-11-24            1.0  0.095104    0.196798  0.003766  0.011299  0.059322
2019-12-01            1.0  0.080963    0.202407  0.003282  0.007659  0.036105
2019-12-08            1.0  0.101207    0.199629  0.015785  0.004643  0.025070
2019-12-15            1.0  0.106134    0.183057  0.011685  0.006816  0.016553
2019-12-22            1.0  0.113546    0.179283  0.005976  0.015936  0.017928
2019-12-29            1.0  0.140052    0.197644  0.009162  0.005236  0.010471
2020-01-05            1.0  0.132924    0.171779  0.007157  0.012270  0.026585
2020-01-12            1.0  0.116392    0.174588  0.005820  0.011639  0.023278
2020-01-19            1.0  0.123418    0.184599  0.004219  0.004219  0.026371
2020-01-26            1.0  0.113527    0.181159  0.007246  0.007246  0.036232

In [92]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 349 entries, 2013-05-26 to 2020-01-26
Freq: W-SUN
Data columns (total 7 columns):
totalArticles    349 non-null int64
Programming      349 non-null int64
Python           349 non-null int64
JavaScript       349 non-null int64
Cpp              349 non-null int64
C                349 non-null int64
Swift            349 non-null int64
dtypes: int64(7)
memory usage: 31.8 KB


In [110]:
new_df[langs] = new_df[langs].div(new_df["totalArticles"], axis=0)

In [111]:
new_df.dropna(0, inplace=True)

In [74]:
p = figure(title="line", plot_width=900, plot_height=450)
p.line(x=new_df.index, y=new_df)

show(p)

In [66]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df', 138672741),
 ('s', 2097376),
 ('new_df', 18640),
 ('v', 2280),
 ('filtered', 1184),
 ('Counter', 1056),
 ('fp', 216),
 ('keys_parent', 184),
 ('drops', 144),
 ('addLanguages', 136),
 ('figure', 136),
 ('get_required_fields', 136),
 ('is_keys_unique', 136),
 ('keys_virtuals', 136),
 ('output_file', 136),
 ('plot', 136),
 ('show', 136),
 ('li', 128),
 ('keys_others', 112),
 ('langs', 104),
 ('pd', 80),
 ('sns', 80),
 ('k', 61),
 ('p', 56),
 ('el', 55),
 ('index', 28),
 ('res', 28)]

In [46]:
s = set()
s.

In [48]:
s = set()
for li in df.loc[:, "tags"].values.tolist():
    for el in li:
        s.add(el)
len(s)

23950

In [52]:
list(s)[1:100]

['Knowledge The Pirate',
 'On Premise',
 'Food Waste',
 'Java Platform',
 'Inovação',
 'Ui Developers',
 'Boxing',
 'Joomla',
 'Westworld',
 'Mario',
 'Qr Code',
 'Brainfuck',
 'Javascript Interview',
 'Manifestation',
 'T Sql',
 'Makers Institute',
 'Crud',
 'Chegg',
 'Blockhubacademy',
 'Qualis',
 'Semantic Satiation',
 'Ponylang',
 'Time Lapse',
 'Hpc',
 'Melonport',
 'Pythontrainingingurgaon',
 'Reflection Week 1',
 'Skip List',
 'Double Pipe',
 'Mapping Tool',
 'Reusable Component',
 'Quality Analyst',
 'Azure Websites',
 'Ocd',
 'Hyperparameter Tuning',
 'Designpattern',
 'Erlang Serpents',
 '13 Year Old',
 'Datatypes',
 'Python Ide',
 'Grpc Gateway',
 'Setstate',
 'Jmap',
 'Minimalism',
 'Generic Netlink',
 'Improve Your Code Quality',
 'Echo Chamber',
 'Python Cheat Sheet',
 'D3plus',
 'Mocks',
 'Estimote',
 'Caffe2',
 'Programs That Write',
 'Synthetic Biology',
 'Sql Server 2012',
 'Travel Event',
 'Black Scholes Model',
 'Complex Function',
 'Information System',
 '스타트업',
 '